In [ ]:
import cv2
from jetbot import Camera, Robot
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading
import urllib.parse

from algorithm import BoxFollower

In [ ]:
# Function to convert BGR8 images to JPEG
def bgr8_to_jpeg(value):
    return cv2.imencode('.jpg', value)[1].tobytes()

try:
    camera = Camera.instance(width=224, height=224)
except:
    pass
robot = Robot()

running = True
paused = False
image = None

In [ ]:
def box_follower_handler():
    print("initialised")
    bf = BoxFollower()
    while running:
        if not paused:
            image = camera.value
            decision = bf.step(image)
            with open("log.txt", "rw") as file:
                file.write(f"Decision: {decision}")

def drive_with_turn_radius(forward_speed, turn_radius, wheel_base):
    """
    Set motor speeds based on forward speed and turn radius, capped to [-0.5, 0.5].

    Parameters:
    - forward_speed: float, desired linear speed (e.g., normalized to [-0.5, 0.5])
    - turn_radius: float or None, desired turn radius (in meters). Use None or float('inf') for straight motion.
    - wheel_base: float, distance between the wheels (in meters)
    """
    if turn_radius is None or abs(turn_radius) == float('inf'):
        # Straight motion
        left_speed = forward_speed
        right_speed = forward_speed
    else:
        # Calculate wheel speeds using differential drive kinematics
        left_speed = forward_speed * (1 - (wheel_base / (2 * turn_radius)))
        right_speed = forward_speed * (1 + (wheel_base / (2 * turn_radius)))

    # Normalize if either speed exceeds absolute cap of 0.5
    max_speed = max(abs(left_speed), abs(right_speed))
    if max_speed > 0.5:
        scale = 0.5 / max_speed
        left_speed *= scale
        right_speed *= scale

    # Cap again to handle possible floating point overflows
    left_speed = max(min(left_speed, 0.5), -0.5)
    right_speed = max(min(right_speed, 0.5), -0.5)

    robot.set_motors(left_speed, right_speed)


# HTTP request handler class
class NetworkHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith('/camera'):
            self.send_response(200)
            self.send_header('Content-type', 'image/jpeg')
            self.end_headers()
            self.wfile.write(bgr8_to_jpeg(camera.value))
        else:
            query = urllib.parse.urlparse(self.path).query
            params = urllib.parse.parse_qs(query)
            if self.path.startswith('/set_motors'):
                left_speed = float(params.get('left', [0])[0])
                right_speed = float(params.get('right', [0])[0])
                robot.set_motors(left_speed, right_speed)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Motors set')
            elif self.path.startswith('/left'):
                speed = float(params.get('speed', [0])[0])
                robot.left(speed)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Left command executed')
            elif self.path.startswith('/right'):
                speed = float(params.get('speed', [0])[0])
                robot.right(speed)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Right command executed')
            elif self.path.startswith('/forward'):
                speed = float(params.get('speed', [0])[0])
                robot.forward(speed)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Forward command executed')
            elif self.path.startswith('/stop'):
                robot.stop()
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Stop command executed')
            elif self.path.startswith('/run_algo'):
                paused = False
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Algorithm started')
            elif self.path.startswith('/stop_algo'):
                paused = True
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Algorithm stopped')
            elif self.path.startswith('/test'):
                drive_with_turn_radius(0.5, 0.5, 0.1)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b'Driving')


# Function to run the HTTP server
def run_server():
    box_follower_thread = threading.Thread(
        target=box_follower_handler, daemon=True)
    box_follower_thread.start()
    
    server = HTTPServer(('0.0.0.0', 8080), NetworkHandler)
    server.serve_forever()

    # Start the server in a separate thread
    # thread = threading.Thread(target=run_server)
    # thread.start()

In [ ]:
run_server()